Работу выполнил Нелюбин Даниил.

Если у вас возникли вопросы по оформелнию или коду пишите на почту danielnelybin@yandex.ru

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly as py
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
py.offline.init_notebook_mode(connected=True)

In [2]:
df_1 = pd.read_csv("data/Trainee_Analyst_Test_DataSet_1.csv")
df_2 = pd.read_csv("data/Trainee_Analyst_Test_DataSet_2.csv")


In [3]:
funnel_tutorial = df_1.groupby(['MaxAchievedStep']).count()
funnel_tutorial_2 = funnel_tutorial.copy(deep=True)
funnel_tutorial.rename(columns={'PlayerUid':"Количество игроков"},inplace=True)
funnel_tutorial.index.name = "Шаг туториала"

Рассмотрим исходные данные.

In [4]:

fig = px.bar(funnel_tutorial,x=funnel_tutorial.index,y='Количество игроков')
fig.update_layout(
    title={
        'text': "Количество игроков остановившихся на шаге туториала",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

По данному графику, можно сделать вывод, что значительная часть людей дошла до конца, но пока нельзя сказать насколько хорошие это цифры, поэтому построим воронку.

In [5]:
temp_arr = np.zeros(funnel_tutorial_2['PlayerUid'].shape)
for idx,i in enumerate(funnel_tutorial_2['PlayerUid']):
    if idx>0:
        temp_arr[idx] = funnel_tutorial_2['PlayerUid'][len(funnel_tutorial_2)-idx-1]+temp_arr[idx-1]
    else:
        temp_arr[idx] = funnel_tutorial_2['PlayerUid'][len(funnel_tutorial_2)-idx-1]

funnel_tutorial_2['PlayerUid'] = temp_arr[::-1]

In [6]:
funnel_tutorial_2.rename(columns={'PlayerUid':"Количество игроков"},inplace=True)
funnel_tutorial_2.index.name = "Шаг туториала"

In [7]:
fig = px.funnel(funnel_tutorial_2,x=funnel_tutorial_2.index,y='Количество игроков')
fig.update_layout(
    title={
        'text': "Воронка туториала",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

По данной воронке туториала можно сделать вывод, что нибольшее количество человек "утекает" на 1, 2 и 6 шагах. Так же видно что по ходу туториала мы теряем больше половины игроков.

In [8]:
labels = ["Процент прошедших далее игроков", "Процент остановившихся игроков"]
funnel_tutorial_2.rename(columns={'RefPlace':"Количество остановившихся игроков"},inplace=True)

fig = make_subplots(2, 4, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],subplot_titles=['0','1','2','3','4','5','6','7'])
fig.add_trace(go.Pie(labels=labels,values=funnel_tutorial_2.loc[0], scalegroup='one',
                     name="Туториал 0"), 1, 1)
fig.add_trace(go.Pie(labels=labels,values=funnel_tutorial_2.loc[1], scalegroup='two',
                     name="Туториал 1"), 1, 2)
fig.add_trace(go.Pie(labels=labels,values=funnel_tutorial_2.loc[2], scalegroup='one',
                     name="Туториал 2"), 1, 3)
fig.add_trace(go.Pie(labels=labels,values=funnel_tutorial_2.loc[3], scalegroup='one',
                     name="Туториал 3"), 1, 4)
fig.add_trace(go.Pie(labels=labels,values=funnel_tutorial_2.loc[4], scalegroup='one',
                     name="Туториал 4"), 2, 1)
fig.add_trace(go.Pie(labels=labels,values=funnel_tutorial_2.loc[5], scalegroup='one',
                     name="Туториал 5"), 2, 2)
fig.add_trace(go.Pie(labels=labels,values=funnel_tutorial_2.loc[6], scalegroup='one',
                     name="Туториал 6"), 2, 3)
fig.add_trace(go.Pie(labels=labels,values=funnel_tutorial_2.loc[7], scalegroup='one',
                     name="Туториал 7"), 2, 4)
fig.update_layout(title_text='Процент прошедших и отсановившихся игроков')
fig.show()

По представленной выше диаграмме можно сделать вывод, что наибольший процент людей останавливается на 0,1 и 6 задании.

#### Далее рассчитаем Retention Day




In [9]:
df_2['DateTime'] =  pd.to_datetime(df_2.DateTime,format="%d.%m.%Y")
df_2['Registered'] =  pd.to_datetime(df_2.Registered,format="%d.%m.%Y")
df_2['Diff'] = df_2.DateTime-df_2.Registered
df_2['Diff'] = df_2['Diff'].apply(lambda x:x.days)
len(df_2[df_2.Diff==1]['PlayerUid'].unique())/len(df_2[df_2.Diff==0]['PlayerUid'].unique())

df_3 = pd.DataFrame({"День 0":[len(df_2[df_2.Diff==0]['PlayerUid'].unique())],"День 1" : [len(df_2[df_2.Diff==1]['PlayerUid'].unique())]})
df_3 = df_3.T
df_3.columns = ['Количество человек']
fig = px.bar(df_3)
fig.update_layout(
    title={
        'text': "Сравнение количества человек зашедших в 0 и 1 дни",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Дни",
    yaxis_title="Количество человек",
    showlegend = False)

fig.show()

In [10]:
len(df_2[df_2.Diff==1]['PlayerUid'].unique())/len(df_2[df_2.Diff==0]['PlayerUid'].unique())

0.08557046979865772

***
### Retention Day 1 = 0.0855
***
